In [2]:
# import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import os

In [3]:
# Specify the path to the directory containing your images
image_dir = "C:\\Users\\Vishaly\\OneDrive - Hong Kong Baptist University\\Miscellaneous\\Vishaly\\Coding\\Data-Science-Projects\\Cleaned-VS-Dirty\\platesv2 (1)\\plates\\__MACOSX\\train"

# Initialize an empty list to hold the loaded images
loaded_images = []

# Loop through each file in the directory
for train in os.listdir(image_dir):
    # Construct the full file path
    file_path = os.path.join(image_dir, train)
    
    # Check if the file is an image (you might want to adjust this depending on your needs)
    if file_path.endswith(( '.jpg')):
        # Read the image using OpenCV
        img = cv2.imread(file_path)
        
        # Add the image to the list
        loaded_images.append(img)

# Now, `loaded_images` contains all your images loaded into memory as NumPy arrays


In [4]:
from PIL import Image
import os

# Assuming your images are stored in a folder named 'images'
image_folder = "C:\\Users\\Vishaly\\OneDrive - Hong Kong Baptist University\\Miscellaneous\\Vishaly\\Coding\\Data-Science-Projects\\Cleaned-VS-Dirty\\platesv2 (1)\\plates\\__MACOSX\\train"

# List to hold all loaded images
images = []

for filename in os.listdir(image_folder):
    if filename.endswith('.jpg'):  # Ensure we're only loading.jpg files
        img_path = os.path.join(image_folder, filename)
        try:
            img = Image.open(img_path)
            images.append(img)
        except IOError:
            print(f"Failed to open {img_path}")

# Now, 'images' contains all your loaded images

In [5]:
# Display all images
for img in images:
    plt.imshow(img)
    plt.show()